# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [10]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """

    >>> priest('male', 40, 24, 0.96, 105, 115, 38.1, 'alert', 'air', 'unrestricted normal activity')
    0.03
    >>> priest('female', 80, 11, 0.93, 105, 95, 40.8, 'alert', 'air', 'limited activity, can self-care')
    0.08
    >>> priest('male', 18, 22, 0.95, 105, 115, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity')
    0.05
    """
    # Convert strings to lowercase for uniformity
    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    # Basic validation for input
    if sex not in ['male', 'female'] or alertness not in ['alert', 'confused or not alert'] or inspired_oxygen not in ['air', 'supplemental oxygen']:
        return None

    # Score calculation logic
    score = 0
    if sex == 'male':
        score += 1
    if age >= 65:
        score += 2
    if respiratory_rate > 20:
        score += 2
    if oxygen_saturation < 0.95:
        score += 2
    if heart_rate > 110:
        score += 1
    if systolic_bp < 90:
        score += 1
    if temperature >= 39.0:
        score += 2
    if alertness == 'confused or not alert':
        score += 2
    if inspired_oxygen == 'supplemental oxygen':
        score += 2
    if 'limited' in performance_status or 'bed/chair' in performance_status:
        score += 2

    # Translate score to risk percentage
    if score >= 26:
        return 0.99
    elif score >= 17:
        return 0.59
    else:
        return round(score / 100, 2)


In [11]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 40, 24, 0.96, 105, 115, 38.1, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.03
ok
Trying:
    priest('female', 80, 11, 0.93, 105, 95, 40.8, 'alert', 'air', 'limited activity, can self-care')
Expecting:
    0.08
ok
Trying:
    priest('male', 18, 22, 0.95, 105, 115, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity')
Expecting:
    0.05
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [12]:
import requests

def find_hospital(age, sex, risk_pct):
    """
    Queries the REST web service to find the appropriate hospital for the patient based on age, sex, and risk percentage.

    Args:
    - age (int): Age of the patient.
    - sex (str): Sex of the patient ('male' or 'female').
    - risk_pct (float): Risk percentage (between 0 and 1).

    Returns:
    - str: Hospital name where the patient should be transferred.

    Returns None if anything fails.

    >>> find_hospital(40, 'male', 0.1) # Simulated expected output
    'Southwest Hospital and Medical Center'
    >>> find_hospital(16, 'female', 0.3) # Simulated expected output
    'Childrens Healthcare of Atlanta at Scottish Rite'
    >>> find_hospital(85, 'female', 0.45) # Simulated expected output
    'Wesley Woods Geriatric Hospital'
    """
    try:
        url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex.lower()}&risk_pct={risk_pct}"
        response = requests.get(url)
        if response.status_code != 200:
            return None
        data = response.json()
        return data['hospital']
    except:
        return None

In [13]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1) # Simulated expected output
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(16, 'female', 0.3) # Simulated expected output
Expecting:
    'Childrens Healthcare of Atlanta at Scottish Rite'
ok
Trying:
    find_hospital(85, 'female', 0.45) # Simulated expected output
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [14]:
import json

def get_address(hospital_name):
    """
    Looks up the hospital address from a JSON file based on the hospital name.

    Args:
    - hospital_name (str): Name of the hospital.

    Returns:
    - str: Address of the hospital.

    Returns None if the hospital is not found.

    >>> get_address('Southwest Hospital and Medical Center')
    '501 FAIRBURN ROAD SW'
    >>> get_address('Childrens Healthcare of Atlanta at Scottish Rite')
    '1001 JOHNSON FERRY RD'
    >>> get_address('Wesley Woods Geriatric Hospital')
    '1821 CLIFTON ROAD, NE'
    """
    try:
        url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
        response = requests.get(url)
        hospitals = json.loads(response.text)
        for hospital in hospitals:
            if hospital['name'].lower() == hospital_name.lower():
                return hospital['address']
        return None
    except:
        return None


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [15]:
def process_people(file_url):
    """
    Processes a pipe-delimited file of patient data, calculates the PRIEST score, finds the hospital, and retrieves the address for each patient.

    Args:
    - file_url (str): URL to the pipe-delimited file.

    Returns:
    - list: A list of dictionaries where each item contains patient data with hospital and address details.
    """
    try:
        response = requests.get(file_url)
        data = response.text.splitlines()
        results = []
        for line in data[1:]:  # Skip the header row
            fields = line.split('|')
            patient_number = fields[0]
            sex = fields[1]
            age = int(fields[2])
            breath = int(fields[3])
            o2_sat = float(fields[4])
            heart = int(fields[5])
            systolic = int(fields[6])
            temp = float(fields[7])
            alertness = fields[8]
            inspired = fields[9]
            status = fields[10]

            # Calculate PRIEST score
            risk = priest(sex, age, breath, o2_sat, heart, systolic, temp, alertness, inspired, status)

            # Find hospital and address
            if risk is not None:
                hospital = find_hospital(age, sex, risk)
                address = get_address(hospital)
                results.append({patient_number: [sex, age, breath, o2_sat, heart, systolic, temp, alertness, inspired, status, hospital, address]})
        return results
    except:
        return []


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [16]:
def check_results(results_url, calculated_results):
    """
    Checks the calculated results against the provided results file.

    Args:
    - results_url (str): URL to the results file in JSON format.
    - calculated_results (list): List of calculated results from process_people().

    Returns:
    - bool: True if the results match, False otherwise.
    """
    try:
        response = requests.get(results_url)
        provided_results = json.loads(response.text)
        return provided_results == calculated_results
    except:
        return False

In [ ]:
file_url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
calculated_results = process_people(file_url)
match = check_results(results_url, calculated_results)
print(f"Results match: {match}")

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---